# Introdução

## Neste Notebook os dados de uma corrida de Kart abaixo serão explorados

O primeiros passo será reescrever esse arquivo em um arquivo CSV com separador ; para carregar os dados em um Dataframe pandas.
O arquivo criado se chama logs.csv

In [1]:
import pandas as pd

In [2]:
df =  pd.read_csv('logs.csv', sep=';')
df.head()

,Hora,Piloto,Nº Volta,Tempo Volta,Velocidade média da volta
0,23:49:08.277,038 – F.MASSA,1,1:02.852,44.275
1,23:49:10.858,033 – R.BARRICHELLO,1,1:04.352,43.243
2,23:49:11.075,002 – K.RAIKKONEN,1,1:04.108,43.408
3,23:49:12.667,023 – M.WEBBER,1,1:04.414,43.202
4,23:49:30.976,015 – F.ALONSO,1,1:18.456,35.470


## Carregamento e transformação dos dados

Criaremos outro dataframe com os dados agregados para cada pilo, nesse dataframe colocaremos todos os campos de interesse i.e. Posição Chegada, Código Piloto, Nome Piloto, Qtde Voltas Completadas e Tempo Total de Prova.

In [3]:
# primeiro, criamos uma lista com o nome e id de cada piloto no dataframe
lista_pilotos_id = df.Piloto.unique()

# agora criamos listas vazias que serão populadas durante o loop
l_nome_piloto = []
l_id_piloto = []
l_n_voltas = []
tempo_total_segundos = []

# loop sob cada piloto, id no dataframe para agregar dados e calcular algumas coisas
# como o tempo total de prova e quantidade de voltas

for piloto_id in lista_pilotos_id:
    
    # do campo piloto tiramos o nome e id do piloto para popular suas respectivas listas
    id, nome = piloto_id.split(' – ')
    l_nome_piloto.append(nome)
    l_id_piloto.append(id)
    
    # do campo N Volta, podemos tirar a quantidade de voltas, que sera o maior numero da lista
    l_n_voltas.append(df[df.Piloto == piloto_id]['Nº Volta'].max())
    
    # Agora vamos somar os tempo de cada volta de cada piloto em segundos para obter o tempo total de prova
    t_total = 0
    for t_volta in df[df.Piloto == piloto_id]['Tempo Volta']:
        t_min, t_seg = t_volta.split(':')
        #vamos transformar tudo para segundos e arredondar para a segunda casa
        t_total+= round(int(t_min)*60 + float(t_seg), 2)
    tempo_total_segundos.append(t_total)


In [4]:
# Agora vamos criar novo dataframe com os campos
# Posição Chegada, Código Piloto, Nome Piloto, Qtde Voltas Completadas e Tempo Total de Prova.
# Posicao Chegada será adicionado em seguida

df2 = pd.DataFrame.from_dict({'Código Piloto': l_id_piloto, 'Nome Piloto': l_nome_piloto,
                             'Qtde Voltas Completadas': l_n_voltas,
                             'Tempo Total de Prova [s]': tempo_total_segundos})

In [5]:
# Vamos criar um dataframe de pilotos que terminaram a corrida
# depois iremos ordenalos por tempo de chegada
# assim iremos adicionar a coluna posicao

df3 = df2[df2['Qtde Voltas Completadas'] == max(l_n_voltas)].sort_values('Tempo Total de Prova [s]', ascending=True)
posicao = []
for i in range(len(df3)):
    posicao.append(i+1)
df3.insert(0, 'Posição Chegada', posicao, True)

# Agora vamos criar um dataframe com todos os pilotos que não terminaram a corrida
# a posicao deles sera igual uma posicao depois do ultimo piloto que terminou a corrida

df2 = df2[df2['Qtde Voltas Completadas'] != max(l_n_voltas)].sort_values('Tempo Total de Prova [s]', ascending=True)
posicao = []
for _ in range(len(df2)):
    posicao.append(i+2)
df2.insert(0, 'Posição Chegada', posicao, True)

df3 = df3.append(df2, sort=False, ignore_index=True)


## Resultado

Posição Chegada, Código Piloto, Nome Piloto, Qtde Voltas Completadas e Tempo Total de Prova podem ser vistos no Dataframe abaixo

In [6]:
df3.head(len(df3))

,Posição Chegada,Código Piloto,Nome Piloto,Qtde Voltas Completadas,Tempo Total de Prova [s]
0,1,038,F.MASSA,4,251.58
1,2,002,K.RAIKKONEN,4,255.16
2,3,033,R.BARRICHELLO,4,256.08
3,4,023,M.WEBBER,4,257.73
4,5,015,F.ALONSO,4,294.22
5,6,011,S.VETTEL,3,387.27


## Explorando um pouco mais

Agora iremos responder a algumas perguntas, são elas:

A melhor volta de cada piloto

A melhor volta da corrida e

Calcular a velocidade média de cada piloto durante toda corrida

#### A melhor volta de cada piloto e a volta mais rápida

In [10]:
df.groupby('Piloto').max()

,Hora,Nº Volta,Tempo Volta,Velocidade média da volta
Piloto,,,,
002 – K.RAIKKONEN,23:52:22.120,4,1:04.108,44.118
011 – S.VETTEL,23:54:57.757,3,3:31.315,35.633
015 – F.ALONSO,23:53:06.741,4,1:20.050,41.528
023 – M.WEBBER,23:52:25.975,4,1:04.805,43.335
033 – R.BARRICHELLO,23:52:22.586,4,1:04.352,43.675
038 – F.MASSA,23:52:17.003,4,1:03.170,44.334


Utilizando o groupby, podemos ver que a melhor volta de todos os pilotos, foi a última de cada um.

Também podemos notar que a volta mais rápida foi a do piloto F.MASSA

#### A velocidade média de cada piloto durante toda corrida

In [8]:
df.groupby('Piloto')['Velocidade média da volta'].mean()

Piloto
002 – K.RAIKKONEN      43.627250
011 – S.VETTEL         25.745667
015 – F.ALONSO         38.066250
023 – M.WEBBER         43.191250
033 – R.BARRICHELLO    43.468000
038 – F.MASSA          44.245750
Name: Velocidade média da volta, dtype: float64

Como era o esperando, F.MASSA foi o piloto com velocidade média mais alta. VETTEL foi o piloto com menor velocidade média.